In [1]:
%load_ext autoreload
%autoreload 2

import lib, data, model
import graphs as gr
 
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

//anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


<h5>Part 1: London Data - cleaning</h5>

In [2]:
# triming the original 56 columns to 17 useful columns
cols = ['bills_included','description',
    'details_url','first_published_date',
    'floor_plan', 'num_bathrooms','num_bedrooms','num_recepts',
    'furnished_state', 'property_type', 'rental_prices.shared_occupancy',
    'latitude', 'longitude','outcode',
    'listing_id','status',
    'rental_prices.per_month']

In [3]:
df = pd.read_csv('./data/london_rental.csv', usecols=cols)

In [4]:
# rename the columns
df.columns = ['bills_included', 'description', 'details_url', 'first_published_date',
          'floor_plan', 'furnished_state', 'latitude', 'listing_id', 'longitude',
          'num_bathrooms', 'num_bedrooms', 'num_recepts', 'outcode',
          'property_type', 'rent_price', 'shared_occu','status'
         ]


In [5]:
df.info()
# data series with null columns are: bills_included, description, floor_plan, furnished_state, property_type


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9900 entries, 0 to 9899
Data columns (total 17 columns):
bills_included          1546 non-null float64
description             9892 non-null object
details_url             9900 non-null object
first_published_date    9900 non-null object
floor_plan              7701 non-null object
furnished_state         8833 non-null object
latitude                9900 non-null float64
listing_id              9900 non-null int64
longitude               9900 non-null float64
num_bathrooms           9900 non-null int64
num_bedrooms            9900 non-null int64
num_recepts             9900 non-null int64
outcode                 9900 non-null object
property_type           8949 non-null object
rent_price              9900 non-null int64
shared_occu             9900 non-null object
status                  9900 non-null object
dtypes: float64(3), int64(5), object(9)
memory usage: 1.3+ MB


In [ ]:

# data series with null columns are: bills_included, description, floor_plan, furnished_state, property_type

# clean Nulls in bills_included
df['bills_included'].fillna(value=0, inplace=True)

# clean Nulls in description
df['description'].fillna(value='No description', inplace=True)

# clean Nulls and make floor_plan a binary column to state if floorplan is available
df['floor_plan'].fillna(value=0, inplace=True)
df['floor_plan'] = df['floor_plan'].map(lambda x: 1 if x != 0 else 0)

# generate a new column indicating student property
df['student'] = df['description'].map(lambda x: 1 if 'student' in x else 0)

# if furnished state is missing and its a student property, assume its furnithsed
df['furnished_state']  = np.where((df['furnished_state'].isna() & df['student'] == 1),
                                'furnished', df['furnished_state'])

df['furnished_state'].fillna(value='Missing', inplace=True)

# turn shared occupency to binary
df['shared_occu'] = df['shared_occu'].map(lambda x: 1 if x == 'Y' else 0)

# after investigation, the 11 bathroom one is an error
df.drop(1835, inplace=True)


# clean up and combine property categories together that means the same thing
def type_classify(x):

  flats = ['Flat','Maisonette']
  terrace = ['Terraced house','End terrace house','Town house','Mews house']
  country = ['Cottage','Detached bungalow','Barn conversion','Semi-detached bungalow']

  if x in flats:
      return 'Flats'
  elif x in terrace:
      return 'Terrace'
  elif x in country:
      return 'Country'
  elif x == np.nan:
      return 'NA'
  else:
      return x

df['property_type'] = df['property_type'].map(lambda x: type_classify(x))

# delete all rows with property type "parking/garage"
indexNames = df[df['property_type'] == 'Parking/garage'].index
df.drop(indexNames, inplace=True)

df['property_type'].fillna(value='Missing', inplace=True)

df['rented'] = df['status'].map(lambda x: 0 if x in ['to_rent'] else 1)
df.drop(['status'], axis=1, inplace=True)

df.drop_duplicates(subset='listing_id', keep='first', inplace=True)